In [1]:
import pandas as pd
import numpy as np
import ipyvolume as ipv
import pyuff
import os

# Importing geometry data

In [2]:
df = pd.read_excel('points.xlsx')#file with geometry data and local CS
cx = np.array([1,0,0])*10
cy = np.array([0,1,0])*10
cz = np.array([0,0,1])*10
x = df['X']
y = df['Y']
z = df['Z']
tm_i = df.keys()[4:-3]

# Matrices for tranformation from global to local CS

In [3]:
trans_matrices = []
j = 0
t = []
for i in tm_i:
    t.append(np.asarray(df[i][:3]))
    j+=1
    if j==3:
        j=0
        t=np.cos(np.transpose(np.asarray(t))*np.pi/180)
        trans_matrices.append(t)
        t=[]

In [4]:
uffwrite = pyuff.UFF('./tree_structure_mini.uff')

# Writing model info

In [5]:
data={'type':151,
      'model_name':'3D tree structure',
      'description':'Dimention: 379x179x474 - CAD model: tree.step',
      'db_app':'0',
      'program':'0'}
uffwrite._write_set(data,'overwrite')

# Writing geometry

In [6]:
data={'type':15,
      'node_nums':np.array(range(len(x))),
      'def_cs':np.zeros_like(x),
      'disp_cs':list(df['cs']),
      'color':np.ones_like(x),
      'x':x,
      'y':y,
      'z':z}
uffwrite._write_set(data,'add')

# Data for trace lines

In [7]:
traces = []
for i in range(1,len(df['cs'])):
    if len(traces)<df['cs'][i]:
        traces.append([])
    traces[df['cs'][i]-1].append(i)

# Writing datasets for each trace line

In [8]:
for i in range(len(traces)):
    data={'type': 82,
     'trace_num': i+1,
     'n_nodes': len(traces[i]),
     'color': 0,
     'id': 'line %i'%(i+1),
     'nodes': np.asarray(traces[i])}
    uffwrite._write_set(data,'add')

# CS matrices to UFF compatible structure

In [9]:
n = len(trans_matrices)
tm = np.zeros([4*n,3])
for i in range(n):
    tm[4*i:4*i+3,:]=trans_matrices[i]
    tm[4*i+3,:]=[0,0,0]

# Writing CS matrices

In [10]:
data={'type':2420,
      'nodes':np.array(range(n)),
      'local_cs':tm}
uffwrite._write_set(data,'add')

In [11]:
n = len(uffwrite.get_set_types())#checking numer of writen datasets

In [12]:
frfs = np.load('FRFs_mini.npy')#importing FRFs data
freq = np.load('Freq_mini.npy')#importing Freq list

# Writing each FRF into own dataset 58

In [13]:
for o in range(3):
    for v in range(3):
        for t in range(43):
            resp_node = 0
            resp_direc = o+1
            ref_node = t+1
            ref_direc = v+1
            frf = frfs[o,v,t,:2000]
            datai={'type':58,
                         'binary':1,
                         'func_type':4, 
                         'rsp_node': resp_node, 
                         'rsp_dir': resp_direc, 
                         'ref_dir': ref_direc, 
                         'ref_node': ref_node,
                         'data': frf,
                         'x': freq,
                         'id1': 'id1', 
                         'rsp_ent_name': 'name',
                         'ref_ent_name': 'name',
                         'abscissa_spacing':1,
                         'abscissa_spec_data_type':18,
                         'ordinate_spec_data_type':12,
                         'orddenom_spec_data_type':13}
            uffwrite._write_set(datai,'add')
            

In [14]:
v_x,v_y,v_z = np.load('shapes.npy')#importing modal shapes
freq = np.load('nat-freq.npy')#importing modal frequences

# Writing each mode into own dataset 55

In [15]:
n=10
if v_x.shape[1]<10:
    n=v_x.shape[1]
for i in range(n):
    vektor_x = v_x[:,i]
    vektor_y = v_y[:,i]
    vektor_z = v_z[:,i]
    data={'type':55,
          'analysis_type':2,
          'data_ch':3,
          'spec_data_type':8,
          'load_case':0,
          'mode_n':i,
          'freq':freq[i],
          'node_nums':np.array(range(1,44)),
          'r1':vektor_x,
          'r2':vektor_y,
          'r3':vektor_z,
          'r4':np.zeros_like(vektor_x),
          'r5':np.zeros_like(vektor_x),
          'r6':np.zeros_like(vektor_x),
         }
    uffwrite._write_set(data,'add')

# Checking number of datasets 55 and 58

In [16]:
j=0
for s in pyuff.UFF('./tree_structure_mini.uff').get_set_types():
    if s==55:
        j+=1

In [17]:
j=0
for s in pyuff.UFF('./tree_structure_mini.uff').get_set_types():
    if s==58:
        j+=1